In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from generations import *
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import matplotlib.pyplot as plt

In [ ]:
data_test = np.load('channel_test.npy')
data_train = np.load('channel_train.npy')
data_test.shape


In [ ]:
def bit_err(y_true, y_pred):
    err = 1 - tf.reduce_mean(
        tf.reduce_mean(
            #tf.to_float(
            float(
                tf.equal(
                    tf.sign(
                        y_pred - 0.5),
                    tf.cast(
                        tf.sign(
                            y_true - 0.5),
                        tf.float32))),
            1))
    return err

# ORIGINAL MODEL

In [ ]:
input_bits = Input(shape=(payloadBits_per_OFDM * 2,))

In [ ]:
temp = BatchNormalization()(input_bits)
temp = Dense(n_hidden_1, activation='relu')(input_bits)
temp = BatchNormalization()(temp)
temp = Dense(n_hidden_2, activation='relu')(temp)
temp = BatchNormalization()(temp)
temp = Dense(n_hidden_3, activation='relu')(temp)
temp = BatchNormalization()(temp)
out_put = Dense(n_output, activation='sigmoid')(temp)
model = Model(input_bits, out_put)
model.compile(optimizer='adam', loss='mse', metrics=[bit_err])
model.summary()
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
model.fit(
    training_gen(100,25), #(1000,25),
    steps_per_epoch=10, #50,
    epochs=500, #10000
    validation_data=validation_gen(100,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
model.load_weights('./temp_trained_25.h5')

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = model.evaluate(
        validation_gen(1000, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')

# HYPERPARAMETER TUNING

## LEARNING RATE (BAYESOPT) 

In [ ]:
def model_builder_lr(hp):
    
    input_bits = Input(shape=(payloadBits_per_OFDM * 2,))
    temp = BatchNormalization()(input_bits)
    temp = Dense(n_hidden_1, activation='relu')(temp) #units = hp.Int(f'units_{1}', min_value=100, max_value=1000,step=100), activation='relu')(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(n_hidden_2, activation='relu')(temp) #units = hp.Int(f'units_{2}', min_value=100, max_value=500,step=100), activation='relu')(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(n_hidden_3, activation='relu')(temp) #units = hp.Int(f'units_{3}', min_value=100, max_value=250,step=50), activation='relu')(temp)
    temp = BatchNormalization()(temp)
    out_put = Dense(n_output, activation='sigmoid')(temp)
    model = Model(input_bits, out_put)
    hp_learning_rate = hp.Choice('learning_rate', values=list(np.logspace(-4,-2,30)))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mse', metrics=[bit_err])
    model.summary()
    return model




In [ ]:
# tuner initialization
tuner_lr = kt.BayesianOptimization(hypermodel = model_builder_lr,
                     objective='val_loss', #factor=3, # factor which you have seen above 
                     max_trials = 50, project_name='BayesOptim')  #,hyperband_iterations=5, project_name='khyperband') # , project_name='hyperband_tuner')

In [ ]:
tuner_lr.search_space_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_bit_err', patience=5)
# Perform hypertuning
tuner_lr.search(training_gen(100,25), epochs=20,steps_per_epoch=10, validation_data=validation_gen(100,25), validation_steps=10,callbacks=[stop_early])
best_hp=tuner_lr.get_best_hyperparameters()[0]


In [ ]:
tuner_lr.results_summary()

In [ ]:
# Build the model with the optimal hyperparameters
callback = tf.keras.callbacks.EarlyStopping(monitor='root_mean_squared_error',
                                              patience=20)
tuned_model = tuner_lr.hypermodel.build(best_hp)
#tuned_model.summary()
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
"""
best_act1 = best_hp.get('activation_1')
best_act2 = best_hp.get('activation_2')
best_drop_rate = best_hp.get('rate1')
best_units1 = best_hp.get('units_1')
best_units2 = best_hp.get('units_2')
#best_l1 = best_hp.get('l1')
#best_l2 = best_hp.get('l2')

print ('activation 1: ',best_act1)
print ('activation 2: ',best_act2)
print('Dropout Rate: ',best_drop_rate)
print('Number of units in hidden layer 1: ',best_units1)
print('Number of units in hidden layer 2: ',best_units1)
#print('l1 Regularization coefficient: ',best_l1)
#print('l2 Regularization coefficient: ',best_l2)
#print('Learning Rate: ',best_learn_rate)
"""
best_learn_rate = best_hp.get('learning_rate')
print('Learning Rate: ',best_learn_rate)

In [ ]:
tuned_model.fit(
    training_gen(100,25), #(1000,25),
    steps_per_epoch=10, #50,
    epochs=500, #10000
    validation_data=validation_gen(100,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = tuned_model.evaluate(
        validation_gen(1000, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')

## HIDDEN LAYERS  1)Random Search   2)Bayesopt

In [ ]:
def model_builder_hl_r(hp):
    input_bits = Input(shape=(payloadBits_per_OFDM * 2,))
    temp = BatchNormalization()(input_bits)
    temp = Dense(units = hp.Int(f'units_{1}', min_value=300, max_value=1500,step=150), activation='relu')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = hp.Int(f'units_{2}', min_value=300, max_value=900,step=150), activation='relu')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = hp.Int(f'units_{3}', min_value=150, max_value=300,step=50), activation='relu')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    out_put = Dense(n_output, activation='sigmoid')(temp)
    model = Model(input_bits, out_put)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.007279), loss='mse', metrics=[bit_err])
    model.summary()
    return model

In [ ]:
# tuner initialization
tuner_hidden_r = kt.RandomSearch(hypermodel = model_builder_hl_r,
                     objective=kt.Objective("val_bit_err", direction="min"), #factor=3, # factor which you have seen above 
                     max_trials = 50, project_name='HiddenLayerRandom')  #,hyperband_iterations=5, project_name='khyperband') # , project_name='hyperband_tuner')

In [ ]:
tuner_hidden_r.search_space_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_bit_err', patience=5)
# Perform hypertuning
tuner_hidden_r.search(training_gen(100,25), epochs=40,steps_per_epoch=10, validation_data=validation_gen(100,25), validation_steps=10,callbacks=[stop_early])
best_hp=tuner_hidden_r.get_best_hyperparameters()[0]


In [ ]:
tuner_hidden_r.results_summary()

In [ ]:
best_units1 = best_hp.get('units_1')
best_units2 = best_hp.get('units_2')
best_units3 = best_hp.get('units_3')
#best_l1 = best_hp.get('l1')
#best_l2 = best_hp.get('l2')

#print ('activation 1: ',best_act1)
#print ('activation 2: ',best_act2)
#print('Dropout Rate: ',best_drop_rate)
print('Number of units in hidden layer 1: ',best_units1)
print('Number of units in hidden layer 2: ',best_units2)
print('Number of units in hidden layer 3: ',best_units3)

In [ ]:
# Build the model with the optimal hyperparameters
callback = tf.keras.callbacks.EarlyStopping(monitor='root_mean_squared_error',
                                              patience=20)
tuned_model_hidden_random = tuner_hidden_r.hypermodel.build(best_hp)
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
tuned_model_hidden_random.fit(
    training_gen(500,25), #(1000,25),
    steps_per_epoch=10, #50,
    epochs=500, #10000
    validation_data=validation_gen(500,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = tuned_model_hidden_random.evaluate(
        validation_gen(1000, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')

##  ACTIVATION FUNC

In [ ]:
def model_builder_act_func(hp):
    input_bits = Input(shape=(payloadBits_per_OFDM * 2,))
    temp = BatchNormalization()(input_bits)
    temp = Dense(units = 1050, activation=hp.Choice('activation_1', values=['selu','elu','relu','mish','tanh']))(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = 600, activation=hp.Choice('activation_2', values=['selu','elu','relu','mish','tanh']))(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = 200, activation=hp.Choice('activation_3', values=['selu','elu','relu','mish','tanh']))(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    out_put = Dense(n_output, activation='sigmoid')(temp)
    model = Model(input_bits, out_put)
    model.compile(optimizer=tf.keras.optimizers.Adam(), #learning_rate=0.007279), 
                  loss='mse', metrics=[bit_err])
    model.summary()
    return model

In [ ]:
# tuner initialization
tuner_activ = kt.BayesianOptimization(hypermodel = model_builder_act_func,
                     objective=kt.Objective("val_bit_err", direction="min"), #factor=3, # factor which you have seen above 
                     max_trials = 50, project_name='ActivationFunc')  #,hyperband_iterations=5, project_name='khyperband') # , project_name='hyperband_tuner')

In [ ]:
tuner_activ.search_space_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_bit_err', patience=5)
# Perform hypertuning
tuner_activ.search(training_gen(200,25), epochs=40,steps_per_epoch=10, validation_data=validation_gen(200,25), validation_steps=10,callbacks=[stop_early])
best_hp=tuner_activ.get_best_hyperparameters()[0]


In [ ]:
tuner_activ.results_summary()

In [ ]:
# Build the model with the optimal hyperparameters
callback = tf.keras.callbacks.EarlyStopping(monitor='root_mean_squared_error',
                                              patience=20)
tuned_model_activ_bayes = tuner_activ.hypermodel.build(best_hp)
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
best_act1 = best_hp.get('activation_1')
best_act2 = best_hp.get('activation_2')
best_act3 = best_hp.get('activation_3')

print ('activation 1: ',best_act1)
print ('activation 2: ',best_act2)
print ('activation 2: ',best_act3)

In [ ]:
tuned_model_activ_bayes.fit(
    training_gen(100,25), #(1000,25),
    steps_per_epoch=10, #50,
    epochs=300, #10000
    validation_data=validation_gen(100,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = tuned_model_activ_bayes.evaluate(
        validation_gen(1000, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')

##  FINAL MODEL

In [ ]:
def model_builder_final():
    input_bits = Input(shape=(payloadBits_per_OFDM * 2,))
    temp = BatchNormalization()(input_bits)
    temp = Dense(units = 1050, activation='tanh')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = 600, activation='mish')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    temp = Dense(units = 200, activation='mish')(temp) #,kernel_regularizer=tf.keras.regularizers.L2(l2=0.0001))(temp)
    temp = BatchNormalization()(temp)
    out_put = Dense(n_output, activation='sigmoid')(temp)
    model = Model(input_bits, out_put)
    model.compile(optimizer=tf.keras.optimizers.Adam(), #learning_rate=0.007279), 
                  loss='mse', metrics=[bit_err])
    model.summary()
    return model

In [ ]:
final_model = model_builder_final()

In [ ]:
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
final_model.fit(
    training_gen(500,25), #(1000,25),
    steps_per_epoch=10, #50,
    epochs=2000, #10000
    validation_data=validation_gen(500,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = final_model.evaluate(
        validation_gen(2000, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')